In [ ]:
#Importing required packages

from googleapiclient.discovery import build
import pandas as pd
from datetime import datetime

In [ ]:
#Creating Objects

youTubeApiKey='your_API_key'
youtube= build("youtube","v3",developerKey=youTubeApiKey) #We’re using YouTube Data API v3

channel_id = 'interested_channel_id'

In [ ]:
statdata = youtube.channels().list(part = "statistics",id = channel_id).execute()
statdata

In [ ]:
#Calling Data from API

statdata = youtube.channels().list(part = "statistics",id = channel_id).execute()
stats = statdata["items"][0]["statistics"]
stats

In [ ]:
#Getting Snippet Data

snippetdata=youtube.channels().list(part="snippet",id=channel_id).execute()
snippetdata

In [ ]:
#Getting Details of all videos

contentdata = youtube.channels().list(id = channel_id, part = 'contentDetails').execute()
playlist_id = contentdata['items'][0]['contentDetails']['relatedPlaylists']['uploads']
videos = [ ]
next_page_token = None
while 1:
    res = youtube.playlistItems().list(playlistId = playlist_id,
                                       part = 'snippet',
                                       maxResults = 50,
                                       pageToken = next_page_token).execute()
    videos += res["items"]
    next_page_token = res.get('nextPageToken')
    if next_page_token is None:
        break

print(videos)

In [ ]:
#Getting Video ID for each Video

video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))
video_ids

In [ ]:
#Getting the statistics of each video

stats = []
for i in range(0, len(video_ids), 40):
    res = (youtube).videos().list(id = ",".join(video_ids[i:i+40]), part = "statistics").execute()
    stats += res["items"]
    print(stats)

In [ ]:
#Collecting All Information in a List & creating a dataframe

title = [ ]
publishedAt = [ ]
liked = [ ]
#disliked = [ ]
views = [ ]
url = [ ]
comment = [ ]

for i in range(len(videos)):
    title.append((videos[i])['snippet']['title'])
    publishedAt.append((videos[i])['snippet']['publishedAt'])
    url.append("https://www.youtube.com/watch?v=" + (videos[i])['snippet']['resourceId']['videoId'])
    liked.append(int((stats[i])['statistics']['likeCount']))
    #disliked.append(int((stats[i])['statistics']['dislikeCount']))
    views.append(int((stats[i])['statistics']['viewCount']))
    comment.append(int((stats[i])['statistics']['commentCount']))

data = {"title":title, "upload date":publishedAt, "url":url,"liked":liked,"views":views,"comment":comment}

df = pd.DataFrame(data)
df